# Dataset Index Generation
Generates indices for train, validation and test sets

In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
from scipy import signal
import matplotlib
#from watchmal.testing.repeating_classifier_training_utils import *
from functools import reduce

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)

sys.path.append("../..")
sys.path.append("..")

from testing.repeating_classifier_training import *

%load_ext autoreload
%matplotlib inline
%autoreload 2

from IPython.display import HTML

In [2]:
original_data_path = "/data/WatChMaL/data/IWCD_mPMT_Short_e-mu-gamma_E0to1000MeV_digihits.h5"
f = h5py.File(original_data_path, "r")

labels = np.array(f['labels'])

## Load dataset

In [3]:
# Import test events from h5 file
original_data_path = "/data/WatChMaL/data/IWCD_mPMT_Short_e-mu-gamma_E0to1000MeV_digihits.h5"
f = h5py.File(original_data_path, "r")

print(f.keys())

hdf5_hit_pmt = f["hit_pmt"]
hdf5_hit_time = f["hit_time"]
hdf5_hit_charge = f["hit_charge"]


hit_pmt = np.memmap(original_data_path, mode="r", shape=hdf5_hit_pmt.shape,
                                    offset=hdf5_hit_pmt.id.get_offset(), dtype=hdf5_hit_pmt.dtype)

hit_time = np.memmap(original_data_path, mode="r", shape=hdf5_hit_time.shape,
                                    offset=hdf5_hit_time.id.get_offset(), dtype=hdf5_hit_time.dtype)

hit_charge = np.memmap(original_data_path, mode="r", shape=hdf5_hit_charge.shape,
                                    offset=hdf5_hit_charge.id.get_offset(), dtype=hdf5_hit_charge.dtype)

angles     = np.array(f['angles'])
energies   = np.array(f['energies'])
positions  = np.array(f['positions'])
labels     = np.array(f['labels'])
root_files = np.array(f['root_files'])

#original_radius = [np.sqrt(original_positions[i,0,0]**2 + original_positions[i,0,2]**2) for i in range(original_positions.shape[0])]


<KeysViewHDF5 ['angles', 'energies', 'event_hits_index', 'event_ids', 'hit_charge', 'hit_pmt', 'hit_time', 'labels', 'positions', 'root_files', 'veto', 'veto2']>


In [4]:
# Set up indices
indices = np.array(range(len(labels)))
# Set up root file set
root_file_set = list(set(root_files))

In [5]:
# Set up dict of file indices
file_dict = {}
for file in root_file_set:
    file_dict[file] = []
print("Dict set")

for idx, root_file in enumerate(root_files):
    file_dict[root_file].append(idx)
print("Done")

Dict set
Done


In [6]:
# Get files associated with each particle type
gamma_indices       = indices[np.where(labels == 0)]
gamma_root_file_set = list(set(root_files[gamma_indices]))

e_indices        = indices[np.where(labels == 1)]
e_root_file_set  = list(set(root_files[e_indices]))

mu_indices       = indices[np.where(labels == 2)]
mu_root_file_set = list(set(root_files[mu_indices]))

In [7]:
# Define indices retrieval function
def get_indices_for_files(file_names):
    all_indices = []
    for file_name in file_names:
        all_indices.extend(file_dict[file_name])
    return np.array(all_indices)
        

In [8]:
mu_test_files, mu_val_files, mu_train_files = mu_root_file_set[0:400], mu_root_file_set[400:500], mu_root_file_set[500:]

mu_test_set, mu_val_set, mu_train_set = get_indices_for_files(mu_test_files), get_indices_for_files(mu_val_files), get_indices_for_files(mu_train_files)

print(mu_test_set)

[17930229 17930230 17930231 ... 18761165 18761166 18761167]


In [9]:
gamma_test_files, gamma_val_files, gamma_train_files = gamma_root_file_set[0:400], gamma_root_file_set[400:500], gamma_root_file_set[500:]

gamma_test_set, gamma_val_set, gamma_train_set = get_indices_for_files(gamma_test_files), get_indices_for_files(gamma_val_files), get_indices_for_files(gamma_train_files)

print(gamma_test_set)

[11941526 11941527 11941528 ... 11259914 11259915 11259916]


In [10]:
e_test_files, e_val_files, e_train_files = e_root_file_set[0:400], e_root_file_set[400:500], e_root_file_set[500:]

e_test_set, e_val_set, e_train_set = get_indices_for_files(e_test_files), get_indices_for_files(e_val_files), get_indices_for_files(e_train_files)

print(e_test_set)

[6781902 6781903 6781904 ... 6843944 6843945 6843946]


In [11]:
# Verify that indices match
all_e_indices = np.concatenate((e_test_set, e_val_set, e_train_set))
print(set(labels[all_e_indices]))

all_gamma_indices = np.concatenate((gamma_test_set, gamma_val_set, gamma_train_set))
print(set(labels[all_gamma_indices]))

all_mu_indices = np.concatenate((mu_test_set, mu_val_set, mu_train_set))
print(set(labels[all_mu_indices]))

{1}
{0}
{2}


In [12]:
# Verify that all events are uniquely accounted for
all_collected_indices = np.concatenate((e_test_set, e_val_set, e_train_set, gamma_test_set, gamma_val_set, gamma_train_set, mu_test_set, mu_val_set, mu_train_set))

print(len(labels))
print(len(all_collected_indices))
print(len(set(all_collected_indices)))

20683927
20683927
20683927


In [13]:
train_idxs = np.concatenate((e_train_set, mu_train_set, gamma_train_set))
val_idxs   = np.concatenate((e_val_set, mu_val_set, gamma_val_set))
test_idxs  = np.concatenate((e_test_set, mu_test_set, gamma_test_set))

In [14]:
np.savez('representative_indices.npz', train_idxs=train_idxs, val_idxs=val_idxs, test_idxs=test_idxs)

In [15]:
#np.savez('equal_indices.npz', train_idxs=train_idxs, val_idxs=val_idxs, test_idxs=test_idxs)